# Ritz, Julia (2010). Using tf-idf-related Measures for Determining the Anaphoricity of Noun Phrases.

```perl
#1st pass
l:=4; #initialize term length l
D:=0; #initialize file counter D

for each Document d i in the corpus
#count document
D++;
p:=1; #initialize character position p
    while p + l in d i
        #sequentially cut into terms t of length l
        t:=substring(d i , p, l);
        #*insert string normalization (optional)*
        #initialize count array where necessary
        C(t, d i ):=0 unless defined;
        #save number of previous mentions
        #(i.e. annotate t with C(t, d i ))
        A(t, d i , p):=C(t, d i );
        #count current mention
        C(t, d i )++;
        #count documents containing t
        #(only on first mention of t)
        E(t)++ if (C(t, d i ) =1);
        p++;
    end; #end while
end; #end for each;


#2nd pass
for each Document d i in the corpus
    for each noun phrase NP s e in d i
        sum:=0; #initialize sum
        #from NP’s starting position. . .
        p:=s;
        #. . . to start of last term
        while p <= e − l + 1
            t:=substring(d i , p, l);
            #*insert string normalization (optional)*
            #get annotation of t at p,
            #calculate tf-idf from it
            #and add it to the current sum
            sum+=(get(t, d i , p)/p)*log(D/E(t));
            #calculate sum of other measures
            ...
        end; #end while

        #average by the number of terms in NP s e
        a:=sum/(e − s − l + 2);
        #annotate sum and means to NP s e
        S(d i , s, e):=sum;
        M (d i , s, e):=a;
    end; #end for each
end; #end for each
```

In [45]:
# from collections import defaultdict

# def first_pass(corpus, term_len=4): # term_len = l
#     """
#     Parameters
#     ----------
#     corpus : list of str
#         a list of 'documents', each represented by a string
#     term_len : int
#         this is the N value for specifying a character ngram (default: 4)

#     Returns
#     -------
#     previous_mentions : dict
#         maps from a (term (str), document ID (int), character position (int)) tuple
#         to the term count (int), i.e. it stores the number of times
#         a term has occured in a document up to the specified character position
#         TODO: why are all term counts zero after the first pass?
#     term_count: defaultdict of defaultdict(int)
#         maps from a term (str) to a dict mapping from document IDs to term counts,
#         i.e. stores the total number of times a term occurred in a document
#     docs_containing_term_count : defaultdict(int)
#         maps from a term (str) to the number of documents it occured in (int)
#     """
#     previous_mentions = {}
#     term_count = defaultdict(lambda : defaultdict(int))
#     docs_containing_term_count = defaultdict(int)

#     for doc_id, doc in enumerate(corpus): # doc_id = D
#         doc_len = len(doc)
#         char_pos = 0 # p 
#         while char_pos+term_len < doc_len: # TODO: check if off-by-one
#             #sequentially cut into terms t of length l
#             term = doc[char_pos:char_pos+term_len] # t
            
#             #*insert string normalization (optional)*            
                
#             #save number of previous mentions
#             previous_mentions[(term, doc_id, char_pos)] = term_count[term][doc_id]
            
#             #count current mention
#             term_count[term][doc_id] += 1
            
#             if term_count[term][doc_id] == 1:
#                 #count documents containing t
#                 #(only on first mention of t)
#                 docs_containing_term_count[term] += 1 # E           
#             char_pos += 1
#     return previous_mentions, term_count, docs_containing_term_count

In [66]:
import discoursegraphs as dg
from collections import defaultdict

def corpus_ngram_frequencies(corpus, term_len=4, term_normalization=lambda x: x): # term_len = l
    """
    calculates ngram frequencies: how often did an ngram occur in a document
    (in total OR up to the character position)?
    
    Parameters
    ----------
    corpus : iterable of DiscourseDocumentGraphs
        a list of documents
    term_len : int
        this is the N value for specifying a character ngram (default: 4)
    term_normalization : function
        a function which normalizes each term.
        (default: identity function; does nothing)

    Returns
    -------
    previous_mentions : defaultdict of defaultdict of defaultdict(int)
        (equivalent to a 3-dimensional array[term][document ID][character position]).
        stores the number of times a term has occured in a document
        up to the specified character position
    term_count: defaultdict of defaultdict(int)
        maps from a term (str) to a dict mapping from document IDs to term counts,
        i.e. stores the total number of times a term occurred in a document
    """
    previous_mentions = defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
    term_count = defaultdict(lambda : defaultdict(int))
    docs_containing_term_count = defaultdict(int)

    for docgraph in corpus:
        doc = dg.get_text(docgraph)
        doc_id = document.name  # doc_id = D
        doc_len = len(document)
        char_pos = 0 # p 
        while char_pos+term_len < doc_len:
            #sequentially cut into terms t of length l
            term = doc[char_pos:char_pos+term_len] # t
            normal_term = term_normalization(term)           
                
            #save number of previous mentions
            previous_mentions[normal_term][doc_id][char_pos] = term_count[normal_term][doc_id]
            
            #count current mention
            term_count[normal_term][doc_id] += 1
         
            char_pos += 1
    return previous_mentions, term_count

In [58]:
def count_docs_containing_term(term_count):
    """
    counts the number of documents a term occured in.
    
    Parameters
    ----------
    term_count: defaultdict of defaultdict(int)
        maps from a term (str) to a dict mapping from document IDs to term counts,
        i.e. stores the total number of times a term occurred in a document
    
    Returns
    -------
    docs_containing_term : dict
        maps from a term (str) to the number of documents it occured in (int)
    """
    return {term:len(docs) for (term, docs) in term_count.iteritems()}

In [64]:
from math import log

def second_pass(corpus, previous_mentions, term_len=4, term_normalization=lambda x: x):
    """
    Parameters
    ----------
    corpus : list of str
        a list of 'documents', each represented by a string
    previous_mentions : defaultdict of defaultdict of defaultdict(int)
        (equivalent to a 3-dimensional array[term][document ID][character position]).
        stores the number of times a term has occured in a document
        up to the specified character position
    term_len : int
        this is the N value for specifying a character ngram (default: 4)
    term_normalization : function
        a function which normalizes each term.
        (default: identity function; does nothing)

    Returns
    -------
    np_sums : dict
        ???
    np_mean : dict
        ???
    """
    doc_term_count = count_docs_containing_term(term_count)
    np_sums = defaultdict(lambda : defaultdict(int))
    np_means = defaultdict(lambda : defaultdict(int))

    for doc_id, doc in enumerate(corpus):
        for np in doc.noun_phrases(): # NP starting at position s and ending at position e
            np_sum = 0
            start_pos = np.startpos()
            end_pos = np.endpos()
            #from NP’s starting position. . .
            char_pos = start_pos
            #. . . to start of last term
            while char_pos <= endpos - term_len + 1: # TODO: off-by-one?
                term = doc[char_pos:char_pos+term_len] # t
                normal_term = term_normalization(term)

                #get annotation of t at p,
                #calculate tf-idf from it
                #and add it to the current sum                
                anno_t_p = previous_mentions[normal_term][doc_id][char_pos]
                np_sum += (anno_t_p / char_pos) * log( len(corpus) / docs_containing_term_count[term])

                # TODO: calculate sum of other measures
                # ...

            #average by the number of terms in NP s e
            average = np_sum / (end_pos - start_pos - term_len + 2) # TODO: off-by-one?
            
            #annotate sum and means to NP s e
            np_sums[doc_id][(start_pos, end_pos)] = np_sum
            np_means[doc_id][(start_pos, end_pos)] = average
    return np_sums, np_means

In [55]:
def nprint(d, tab=0, tab_width=2):
    '''print nested key-value datastructures (e.g. dicts)'''
    for k, v in d.iteritems():
        if not hasattr(v, 'iteritems'):
            print '{}{} {}'.format(' '*tab, k, v)
        else:
            print '{}{}:'.format(' '*tab, k)
            nprint(v, tab=tab+tab_width)

In [56]:
prev, term_count, doc_count = first_pass(['haushaushaushaus'])

In [49]:
prev, term_count, doc_count = first_pass(['das erste haus ist rot. das zweite haus ist auch rot.'])

In [50]:
prev, term_count, doc_count = first_pass(['die roten haeuser. die toten haeuser.'])

In [51]:
prev, term_count, doc_count = first_pass(['die toten haeuser.', 
                                          'die boten von gruenau.'])

In [52]:
prev, term_count, doc_count = first_pass(['die roten haeuser. die toten haeuser.', 
                                          'die boten von gruenau.'])

In [53]:
prev, term_count, doc_count = first_pass(['das erste haus ist rot. das zweite haus ist auch rot.', 
                                          'die roten haeuser. die haeuser sind rot. die toten haeuser.', 
                                          'der turm ist gruen.',
                                          'zwei haeuser sind auch rot.'
                                          'die boten von gruenau.'])

In [13]:
# for term in term_count:
#     print term, term_count[term]

# for term in doc_count:
#     print "term '{}' is in {} document(s)".format(term, doc_count[term])

# TODO: how to create a strongly nested defaultdict?

## ALTERNATIVE: use nested numpy arrays?

In [36]:
previous_mentions = defaultdict(lambda : defaultdict(int))
previous_mentions['a']['b']['c'] = 23

TypeError: 'int' object does not support item assignment

In [38]:
previous_mentions['a']['b'] = 23

In [39]:
previous_mentions['a']['b']['c']

TypeError: 'int' object has no attribute '__getitem__'

In [40]:
turtles = defaultdict(lambda : defaultdict(lambda : defaultdict(int)))

In [44]:
turtles[1][2][3][4]

TypeError: 'int' object has no attribute '__getitem__'